## SOI Localization Estimate

In [1]:
from   ipywidgets import interactive, fixed
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from   pathlib import Path
from   PIL import Image
from   utils.evaluation import evaluate_object_detection, evaluate_soi_localization, get_annotated_slices, get_detections, get_ground_truth_bboxes, get_ground_truth_slices, get_scores_and_bboxes
from   utils.metrics import IoU
from   utils.sets import difference, intersection, union

### Setup

In [2]:
annotation_file = 'my_annotations.csv'
results_dir     = 'results'

### Grount truth (annotations)

CT slices which have been annotated with a bounding box and a class label

In [3]:
df_annotated_slices = get_annotated_slices(annotation_file)
df_annotated_slices

,image_path,x1,y1,x2,y2,class_name,ct_id
177,data/ct_scan_0/ae_0_0_3_s_258.jpg,216.0,230.0,276.0,259.0,sternum,ct_scan_0
178,data/ct_scan_0/ae_0_0_3_s_259.jpg,214.0,231.0,277.0,259.0,sternum,ct_scan_0
483,data/ct_scan_1/ae_1_1_3_s_238.jpg,216.0,241.0,289.0,267.0,sternum,ct_scan_1
692,data/ct_scan_2/ae_10_0_9_s_70.jpg,212.0,194.0,306.0,233.0,sternum,ct_scan_2


### Predictions (RetinaNet detections)

CT slices for which the RetinaNet has predicted a bounding box and class label

In [4]:
df_detections = get_detections(results_dir, df_annotated_slices)
df_detections

,retina_net_detections,ct_id
0,results/ct_scan_0/ct_scan_0_predictions.csv,ct_scan_0
1,results/ct_scan_1/ct_scan_1_predictions.csv,ct_scan_1
2,results/ct_scan_2/ct_scan_2_predictions.csv,ct_scan_2


### Obejct Detection

In [5]:
evaluate_object_detection(df_detections, df_annotated_slices, print_confusion_matrix=True, print_scans=True)

Found predictions for n=3 CT scans

************************************
Scan: ct_scan_0
	True slices:     [258 259]
	Detected slices: [258 259]
	IoUs:            [0.73191611 0.865768  ]

Confusion matrix:
	n_slices     = 328
	n_annots     = 2
	# TP         = 2/2 (100.00 %)
	# TN         = 326/326 (100.00 %)
	# FP         = 0/326 (0.00 %)
	# FN         = 0/2 (0.00 %)

************************************
Scan: ct_scan_1
	True slices:     [238]
	Detected slices: [238]
	IoUs:            [0.79716465]

Confusion matrix:
	n_slices     = 296
	n_annots     = 1
	# TP         = 1/1 (100.00 %)
	# TN         = 295/295 (100.00 %)
	# FP         = 0/295 (0.00 %)
	# FN         = 0/1 (0.00 %)

************************************
Scan: ct_scan_2
	True slices:     [70]
	Detected slices: [70]
	IoUs:            [0.97592318]

Confusion matrix:
	n_slices     = 93
	n_annots     = 1
	# TP         = 1/1 (100.00 %)
	# TN         = 92/92 (100.00 %)
	# FP         = 0/92 (0.00 %)
	# FN         = 0/1 (0.00 %)

***

### Structure-Of-Interest (SOI) Localization

In [6]:
evaluate_soi_localization(df_detections, df_annotated_slices, print_scans=True)

Found predictions for n=3 CT scans

************************************
Scan: ct_scan_0
	True slices:      [258 259]
	Localized slices: [259]

************************************
Scan: ct_scan_1
	True slices:      [238]
	Localized slices: [238]

************************************
Scan: ct_scan_2
	True slices:      [70]
	Localized slices: [70]

************************************
Summary:

Confusion matrix:
	#TP = 3
	#FP = 0
	#FN = 0

Accuracy = 3/3 (100.00 %)

Localization distance:
	Average localization distance in xy-plane for TPs: 5.2 +/- 3.4 pxl (n=3)
